In [ ]:
import pandas as pd
import uuid
import environment as env
from utils import *

In [ ]:
current_tenant = get_tenant(env.current_shopping)
current_tenant

In [ ]:
transactions = pd.read_pickle('./data/transaction.pickle')
transactions = transactions.query("shopping_id == @env.current_shopping")
transactions = transactions.reset_index(drop=True)
transactions

In [ ]:
cards = pd.read_pickle('./data/card.pickle')
cards = cards.query("shopping_id == @env.current_shopping")
cards = cards.reset_index(drop=True)
cards

In [ ]:
receipts = pd.DataFrame([], columns=['id', 'shopping_id', 'customer_id', 'transaction_id', 'ticket_id', 'card_id', 'rps', 'credit_card_brand', 'credit_card_expiration_date', 'credit_card_holder', 'credit_card_name', 'credit_card_number'])

for transaction in transactions.itertuples():
    card = cards.loc[cards['id'] == transaction.card_id]
    card = card if card.size != 0 else None
    
    newReceipt = {
        'id': uuid.uuid4(),
        'shopping_id': transaction.shopping_id,
        'customer_id': transaction.customer_id,
        'transaction_id': transaction.id,
        'ticket_id': transaction.ticket_id,
        'card_id': transaction.card_id if bool(transaction.card_id) else None,
        'rps': None,
        'credit_card_brand': card['brand'].iloc[0] if card is not None else None,
        'credit_card_expiration_date': card['expiration_date'].iloc[0] if card is not None else None,
        'credit_card_holder': card['holder'].iloc[0] if card is not None else None,
        'credit_card_name': card['name'].iloc[0] if card is not None else None,
        'credit_card_number': card['number'].iloc[0] if card is not None else None
    }

    receipts.loc[len(receipts)] = newReceipt

receipts

In [ ]:
receipts.to_pickle('./data/receipt.pickle')